In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fra-eng/fra.txt
/kaggle/input/hin-eng/hin.txt
/kaggle/input/hindi-eng/hindi_english_parallel.csv


In [2]:
lines= pd.read_table('/kaggle/input/fra-eng/fra.txt', names=['eng', 'fra'],index_col=False,nrows = 200000)

/tmp/ipykernel_29/3470684711.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  lines= pd.read_table('/kaggle/input/fra-eng/fra.txt', names=['eng', 'fra'],index_col=False,nrows = 200000)


In [3]:
lines.eng[:5],lines.fra[:5]

(0    Go.
 1    Go.
 2    Go.
 3    Go.
 4    Hi.
 Name: eng, dtype: object,
 0          Va !
 1       Marche.
 2    En route !
 3       Bouge !
 4       Salut !
 Name: fra, dtype: object)

In [4]:
lines.shape

(200000, 2)

In [5]:
import string
lines.eng = lines.eng.apply(lambda x:x.lower())
lines.fra = lines.fra.apply(lambda x:x.lower())
punctuation = set(string.punctuation)
lines.eng = lines.eng.apply(lambda x:''.join(char for char in x if char not in punctuation))
lines.fra = lines.fra.apply(lambda x:''.join(char for char in x if char not in punctuation))

In [6]:
lines.eng[10000],lines.fra[10000]

('what a fiasco', 'quel échec\xa0')

In [7]:
digits = set(string.digits)
lines.eng = lines.eng.apply(lambda x:''.join(char for char in x if char not in digits))
lines.fra = lines.fra.apply(lambda x:''.join(char for char in x if char not in digits))

In [8]:
import re
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.fra=lines.fra.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.fra=lines.fra.apply(lambda x: re.sub(" +", " ", x))

In [9]:
lines.fra = lines.fra.apply(lambda x : '<SOS> '+ x + ' <EOS>')

In [10]:
lines.fra[:10]

0                              <SOS> va <EOS>
1                          <SOS> marche <EOS>
2                        <SOS> en route <EOS>
3                           <SOS> bouge <EOS>
4                           <SOS> salut <EOS>
5                           <SOS> salut <EOS>
6                           <SOS> cours <EOS>
7                          <SOS> courez <EOS>
8    <SOS> prenez vos jambes à vos cous <EOS>
9                            <SOS> file <EOS>
Name: fra, dtype: object

In [11]:
from sklearn.utils import shuffle
lines = shuffle(lines)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
lines.sample(5)

,eng,fra
92228,i will choose one of them,<SOS> je choisirai un de ceuxlà <EOS>
46472,can you make a salad,<SOS> peuxtu préparer une salade <EOS>
199763,you shouldnt let people use you like that,<SOS> tu ne devrais pas laisser les gens tutil...
30980,why are you angry,<SOS> pourquoi estu en colère <EOS>
9145,symptoms vary,<SOS> les symptômes sont variables <EOS>


**Let's find total number of english and french words**

In [13]:
english_words = set()
for line in lines.eng:
    for word in line.split():
        if word not in english_words:
            english_words.add(word)
french_words = set()
for line in lines.fra:
    for word in line.split():
        if word not in french_words:
            french_words.add(word)            

In [14]:
total_english_words = len(english_words)
total_french_words = len(french_words)
total_english_words,total_french_words

(13720, 28302)

**To find the maximum sentence length of english and french sentence**

In [15]:
import numpy as np
eng_len = []
for line in lines.eng:
    eng_len.append(len(line.split()))
max_english_sen_len = np.max(eng_len)
max_english_sen_len

12

In [16]:
fra_len = []
for line in lines.fra:
    fra_len.append(len(line.split()))
max_french_sen_len = np.max(fra_len)
max_french_sen_len

22

In [17]:
input_words = sorted(list(english_words))
target_words = sorted(list(french_words))
num_encoder_tokens = len(english_words)+1
num_decoder_tokens = len(french_words)+1
num_decoder_tokens+=1 #zero padding
num_encoder_tokens, num_decoder_tokens

(13721, 28304)

In [18]:
from sklearn.model_selection import train_test_split
x, y = lines.eng, lines.fra
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
x_train.shape, x_test.shape

((160000,), (40000,))

In [19]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [20]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

# **Seq2Seq model(encoder and decoder)**

In [21]:
from keras.layers import Dense,LSTM,Embedding,Input,Bidirectional,Concatenate
from keras.models import Model

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [22]:
latent_dimension = 100

In [23]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=2, min_delta = 0.01)

In [24]:
def generate_batch(X, Y, batch_size=128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_english_sen_len), dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_french_sen_len), dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_french_sen_len, num_decoder_tokens), dtype='float32')
            
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], Y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word]
                for t, word in enumerate(target_text.split()):
                    if t < len(target_text.split()) - 1:
                        decoder_input_data[i, t] = target_token_index[word]
                    if t > 0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            
            yield ([encoder_input_data, decoder_input_data], decoder_target_data)
batch_size = 128
train_gen = generate_batch(x_train, y_train, batch_size)
num_batches_to_print = 2
for batch_num in range(num_batches_to_print):
    [encoder_input_batch, decoder_input_batch], decoder_target_batch = next(train_gen)
    print(f"Batch {batch_num + 1}:")
    print("Encoder Input Batch:")
    print(encoder_input_batch)
    print("Decoder Input Batch:")
    print(decoder_input_batch)
    print("Decoder Target Batch:")
    print(decoder_target_batch)
    print("-" * 30)
    print(encoder_input_batch.shape,decoder_input_batch.shape,decoder_target_batch.shape)

Batch 1:
Encoder Input Batch:
[[ 5981.  7067.  4843. ...     0.     0.     0.]
 [12148.  8436.  8117. ...     0.     0.     0.]
 [ 4167.  7071. 12726. ...     0.     0.     0.]
 ...
 [12454.  6957. 13561. ...     0.     0.     0.]
 [ 3571. 13681.  5584. ...     0.     0.     0.]
 [ 3587.   500.  5584. ...     0.     0.     0.]]
Decoder Input Batch:
[[2.0000e+00 1.2662e+04 1.5461e+04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.0000e+00 3.3400e+03 1.8768e+04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.0000e+00 2.5978e+04 1.3659e+04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [2.0000e+00 1.3799e+04 9.8490e+03 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.0000e+00 6.7030e+03 8.4000e+03 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.0000e+00 9.2310e+03 2.0474e+04 ... 0.0000e+00 0.0000e+00 0.0000e+00]]
Decoder Target Batch:
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 

In [25]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the encoder
encoder_inputs = Input(shape=(max_english_sen_len,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dimension, mask_zero=True)(encoder_inputs)
encoder_lstm1 = LSTM(latent_dimension, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2)
encoder_outputs1, state_h1, state_c1 = encoder_lstm1(encoder_embedding)
encoder_lstm2 = LSTM(latent_dimension, return_state=True, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
encoder_outputs2, state_h2, state_c2 = encoder_lstm2(encoder_outputs1)
encoder_lstm3 = LSTM(latent_dimension, return_state=True, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
encoder_outputs, state_h, state_c = encoder_lstm2(encoder_outputs2)

encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(max_french_sen_len,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dimension, mask_zero=True)(decoder_inputs)
decoder_lstm1 = LSTM(latent_dimension, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2)
decoder_lstm2 = LSTM(latent_dimension, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2)
decoder_lstm3 = LSTM(latent_dimension, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2)

decoder_outputs1, _, _ = decoder_lstm1(decoder_embedding, initial_state=encoder_states)
decoder_outputs2, _, _ = decoder_lstm2(decoder_outputs1)
decoder_outputs, state_h, state_c = decoder_lstm3(decoder_outputs1)

#Attention mechanism
attention = Attention(use_scale=True, name='attention_layer')
context_vector = attention([decoder_outputs, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context_vector, decoder_outputs])

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_combined_context)

# Define the model
seq2seq_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
adam_optimizer = Adam(learning_rate = 0.01)
# Compile the model
seq2seq_model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
seq2seq_model.summary()

# Train the model using the data generator
batch_size = 128
train_gen = generate_batch(x_train, y_train, batch_size)
test_gen = generate_batch(x_test, y_test, batch_size)
seq2seq_model.fit(train_gen, steps_per_epoch=len(x_train)//batch_size, epochs=50,validation_data = test_gen, validation_steps = len(x_test)//batch_size,callbacks=[early_stopping])


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 12)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 12, 100)      1372100     ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 12, 100),    80400       ['embedding[0][0]']              
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                              

In [29]:
x_train,y_train

(191679    i look forward to meeting you again soon
 158969           that pair of pliers came in handy
 116449                everyone looks uncomfortable
 126786               i hope that you get well soon
 184501        if i find your passport ill call you
                             ...                   
 7295                                  dont do that
 52809                         we have an advantage
 997                                      get going
 44732                           we dont understand
 112014                 she advised him to exercise
 Name: eng, Length: 160000, dtype: object,
 191679     <SOS> je me réjouis de vous revoir bientôt <EOS>
 158969                 <SOS> cette pince savéra utile <EOS>
 116449         <SOS> tout le monde a lair mal à laise <EOS>
 126786     <SOS> jespère que tu te rétabliras bientôt <EOS>
 184501    <SOS> si je trouve votre passeport je vous app...
                                 ...                        
 7295              

In [30]:
seq2seq_model.save_weights('/kaggle/working/nmt_weights.h5')
seq2seq_model.save('/kaggle/working/nmt.h5')

# **Translate input**

In [33]:
def translate(input_sentence):
    # Preprocess input sentence
    input_sentence = input_sentence.lower()
    input_sentence = ''.join(char for char in input_sentence if char not in punctuation)
    input_sentence = ''.join(char for char in input_sentence if char not in digits)
    input_sentence = input_sentence.strip()
    input_sentence = re.sub(" +", " ", input_sentence)
    
    # Tokenize input sentence
    input_words = input_sentence.split()
    input_indices = [input_token_index.get(word, 0) for word in input_words]
    print(input_indices)
    # Create encoder input
    encoder_input_data = np.zeros((1, max_english_sen_len), dtype='float32')
    for t, index in enumerate(input_indices):
        encoder_input_data[0, t] = index
    
    # Initial decoder input
    decoder_input_data = np.zeros((1, 1), dtype='float32')
    decoder_input_data[0, 0] = target_token_index['<SOS>']
    
    # Translate using the model
    translation = ''
    for _ in range(max_french_sen_len):
        decoder_output = seq2seq_model.predict([encoder_input_data, decoder_input_data])
        predicted_token_index = np.argmax(decoder_output[0, -1, :])
        print(predicted_token_index)
        predicted_word = reverse_target_char_index.get(predicted_token_index, '<UNK>')
        
        if predicted_word == '<EOS>':
            break
        
        translation += predicted_word + ' '
        
        # Update decoder input for next iteration
        decoder_input_data = np.zeros((1, 1), dtype='float32')
        decoder_input_data[0, 0] = predicted_token_index
    
    return translation.strip()

input_sentence = "i am sad"
predicted_translation = translate(input_sentence)
print("Input:", input_sentence)
print("Predicted Translation:", predicted_translation)

[5981, 375, 10220]
1/1 [==============================] - 0s 34ms/step
12662
1/1 [==============================] - 0s 33ms/step
24578
1/1 [==============================] - 0s 34ms/step
26288
1/1 [==============================] - 0s 33ms/step
12662
1/1 [==============================] - 0s 38ms/step
24578
1/1 [==============================] - 0s 34ms/step
26288
1/1 [==============================] - 0s 33ms/step
12662
1/1 [==============================] - 0s 35ms/step
24578
1/1 [==============================] - 0s 34ms/step
26288
1/1 [==============================] - 0s 34ms/step
12662
1/1 [==============================] - 0s 34ms/step
24578
1/1 [==============================] - 0s 36ms/step
26288
1/1 [==============================] - 0s 35ms/step
12662
1/1 [==============================] - 0s 34ms/step
24578
1/1 [==============================] - 0s 34ms/step
26288
1/1 [==============================] - 0s 34ms/step
12662
1/1 [==============================] - 0s 35ms/step
2